In [19]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.53


In [20]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: crypto_prediction_ws
Azure region: canadacentral
Resource group: crypto_prediction


In [21]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                                max_nodes=6)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

Found existing compute target.
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2019-08-02T03:27:17.296000+00:00', 'errors': None, 'creationTime': '2019-08-02T01:56:40.113211+00:00', 'modifiedTime': '2019-08-02T01:57:27.031652+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 6, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


## Create a project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [4]:
import os
project_folder = './pytorch-price_prediction'
os.makedirs(project_folder, exist_ok=True)

In [22]:
from azureml.core import Experiment

experiment_name = 'pytorch1-price_prediction'
experiment = Experiment(ws, name=experiment_name)

In [29]:
from azureml.train.dnn import PyTorch

estimator = PyTorch(source_directory=project_folder, 
                    script_params={'--serve': 'True','--output-dir': './outputs'},
                    compute_target=compute_target,
                    entry_script='train.py',
                    use_gpu=True)

# upgrade to PyTorch 1.0 Preview, which has better support for ONNX
estimator.conda_dependencies.remove_conda_package('pytorch=1.1.0')
#estimator.conda_dependencies.add_conda_package('pytorch-nightly')
estimator.conda_dependencies.add_channel('pytorch')

WARNING - framework_version is not specified, defaulting to version 1.1.


In [30]:
estimator

In [ ]:
%%time
run.wait_for_completion(show_output=True)
# run = experiment.submit(estimator)
# print(run.get_details())

# from azureml.widgets import RunDetails
# RunDetails(run).show()


RunId: pytorch1-price_prediction_1564719915_fefd9a68
Web View: https://mlworkspace.azure.ai/portal/subscriptions/509b3593-2de3-40ce-a6b8-a838635aecb6/resourceGroups/crypto_prediction/providers/Microsoft.MachineLearningServices/workspaces/crypto_prediction_ws/experiments/pytorch1-price_prediction/runs/pytorch1-price_prediction_1564719915_fefd9a68


In [39]:
# list all the files from the run
run.get_file_names()

[]

In [38]:
model_path = os.path.join('outputs', 'price_predictor.onnx')
run.download_file(model_path, output_file_path=model_path)

UserErrorException: File with path outputs/price_predictor.onnx was not found,
available files include: .

In [40]:
model = run.register_model(model_name='price_predictor', model_path=model_path)
print(model.name, model.id, model.version, sep = '\t')

ModelPathNotFoundException: Could not locate the provided model_path outputs/price_predictor.onnx in the set of files uploaded to the run: []
                See https://aka.ms/run-logging for more details.

## Scoring File

In [11]:
%%writefile score.py
import json
import time
import sys
import os
from azureml.core.model import Model
import numpy as np    # we're going to use numpy to process input and output data
import onnxruntime    # to inference ONNX models, we use the ONNX Runtime
import torch
import torch.nn as nn

class Inferencer(object):
    def __init__(self):
        self.model = self.open_model()
    
    def open_model(self):
        model = TimeRNN(bat_size=1,in_features=3,h_size=1,layer_amnt=1)
        model.load_state_dict(torch.load(config['model_save_loc']))
        model.eval()
        return model

    def un_normalize(self,norm_val,min_val,max_val,typelist=None):
        if(typelist):
            for idx,item in enumerate(norm_val):
                new_val = item * (max_val - min_val) + min_val
                norm_val[idx] = new_val
            return norm_val
        else:
            return norm_val * (max_val - min_val) + min_val 

    def inference(self,value, normalize_method, model,minimum_price,maximum_price):
        value = np.array(value)
        predictions = []
        for sample in value:
            sample = np.array(sample).reshape(1,-1)
            example = torch.tensor(normalize_method.transform(sample)).float()
            
            if(str(device) == 'cuda'):
                example = example.to(device)

            output = model(example)
            output_unnorm = self.un_normalize(norm_val=output.detach(),min_val=minimum_price,max_val=maximum_price)
            predictions.append(output_unnorm)
        return predictions

    def fetch_latest_BTC_JSON(self):
        """Fetch the latest JSON data"""
        API_LINK = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey=SAITMI5ZUMGEKGKY'
        page = requests.get(API_LINK).json()
        return page
    def parse_alphaV_JSON(self,raw_data):
        # Remove meta data for now
        raw_data.pop('Meta Data',None)
        # Remove key name
        df = pd.DataFrame.from_dict(raw_data['Time Series (Digital Currency Daily)'],dtype=float)
        # Flip dates as columns into rows
        df = df.transpose()
        return df


def init():
    global session
    model = Model.get_model_path(model_name='price_predictor')
    session = onnxruntime.InferenceSession(model)

def preprocess(input_data_json):
    # convert the JSON data into the tensor input
    return np.array(json.loads(input_data_json)['data']).astype('float32')

def postprocess(result):
    
    inf = Inferencer()
    
    raw_data = inf.fetch_latest_BTC_JSON()
    df = inf.parse_alphaV_JSON(raw_data=raw_data)
    prices = np.array(df['4a. close (USD)'].tolist())
    data_df_temp = df.drop(labels=['1a. open (USD)','1b. open (USD)','2b. high (USD)','3b. low (USD)','4a. close (USD)','4b. close (USD)','6. market cap (USD)'],axis=1)
    minmax_2 = preprocessing.MinMaxScaler()
    data_df_temp = pd.DataFrame(minmax_2.fit_transform(data_df_temp), columns=data_df_temp.columns)

    minimum_price = np.min(prices)
    maximum_price = np.max(prices)   
    res = un_normalize(norm_val=result,min_val=minimum_price,max_val=maximum_price)
    
    return res

def run(input_data_json):
    try:
        start = time.time()   # start timer
        input_data = preprocess(input_data_json)
        input_name = session.get_inputs()[0].name  # get the id of the first input of the model   
        result = session.run([], {input_name: input_data})
        end = time.time()     # stop timer
        return {"result": postprocess(result),"time": end - start}
    except Exception as e:
        result = str(e)
        return {"error": result}

Writing score.py


## Create container image

In [12]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(pip_packages=["numpy","onnxruntime","azureml-core","torch","pyyaml","pandas","scikit-learn"])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [15]:

from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  docker_file = "Dockerfile",
                                                  description = "Price Prediction ONNX",
                                                  tags = {"demo": "onnx"}
                                                 )


image = ContainerImage.create(name = "onnxpriceprediction",
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

NameError: name 'model' is not defined